# Importing data


In [1]:
import pandas as pd
df1 = pd.read_csv('USA_Topic_Modelling.csv') #Put the path where the extracted data has been stored

In [2]:
#creating temporary Dataframe
df = pd.DataFrame()
df['State'] = df1['State']
df['Party'] = df1['Party']
df['Year'] = df1['Year']
df['Manifesto'] = df1['Manifesto']

In [3]:
df

,State,Party,Year,Manifesto
0,Alabama,Alabama Democratic Party,2008,Alabama Democratic Party PlatformTHE DEMOCRATI...
1,Alaska,Alaska Democratic Party,2008,Alaska Democratic Party 2008 Platform Moving A...
2,Alaska,Alaska Republican Party,2008,2008 Alaska GOP Platform Mission StatementThe...
3,Arizona,Arizona Democratic Party,2008,Arizona Democratic Party PlatformArizona Democ...
4,Arizona,Arizona Republican Party,2008,"Arizona Republican Party, 2007-2008 State Plat..."
...,...,...,...,...
322,WI,GOP,2006,2006 Republican Party of Wisconsin Platform ...
323,WI,GOP,2006,2006 Wyoming Republican Party Resolutions Wy...
324,NC,GOP,2007,North Carolina Republican Party Platform 2007...
325,WI,Dem,2007,2007 DEMOCRATIC PARTY OF WISCONSIN RESOLUTIONS...


## Importing necessary nltk tools

In [4]:
from nltk.tokenize import sent_tokenize

from tqdm import tqdm_notebook
import nltk
tqdm_notebook().pandas()

<ipython-input-4-ab3bc79422cc>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


### Making a dataframe of only manifesto

In [5]:
d = pd.DataFrame()
d['Manifesto'] = df['Manifesto'] 

In [6]:
#sentence tokenization
d['Sentences'] = d.Manifesto.progress_map(sent_tokenize)
d['Sentences'].head(1).tolist()[0][:3]

['Alabama Democratic Party PlatformTHE DEMOCRATIC PARTY HOLDS TO THESE PRINCIPLES:That government functions best when it is closest to the people.That Democrats are determined to provide economic opportunity for all Alabamians.',
 'It is our purpose to take maximum advantage of the bountiful natural resources with which we are blessed.',
 'We work for the day when no young Alabamian must go beyond the confines of our State to find opportunity.']

# Text preprocessing

In [7]:
#word tokenization
from nltk.tokenize import word_tokenize
d['tokens_sentences'] = d['Sentences'].progress_map(lambda Sentences: [word_tokenize(sentence) for sentence in Sentences])
print(d['tokens_sentences'].head(1).tolist()[0][:3])



[['Alabama', 'Democratic', 'Party', 'PlatformTHE', 'DEMOCRATIC', 'PARTY', 'HOLDS', 'TO', 'THESE', 'PRINCIPLES', ':', 'That', 'government', 'functions', 'best', 'when', 'it', 'is', 'closest', 'to', 'the', 'people.That', 'Democrats', 'are', 'determined', 'to', 'provide', 'economic', 'opportunity', 'for', 'all', 'Alabamians', '.'], ['It', 'is', 'our', 'purpose', 'to', 'take', 'maximum', 'advantage', 'of', 'the', 'bountiful', 'natural', 'resources', 'with', 'which', 'we', 'are', 'blessed', '.'], ['We', 'work', 'for', 'the', 'day', 'when', 'no', 'young', 'Alabamian', 'must', 'go', 'beyond', 'the', 'confines', 'of', 'our', 'State', 'to', 'find', 'opportunity', '.']]


In [8]:
#creating POS tags for tokenized words
from nltk import pos_tag
d['POS_tokens'] = d['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(d['POS_tokens'].head(1).tolist()[0][:3])


[[('Alabama', 'NNP'), ('Democratic', 'NNP'), ('Party', 'NNP'), ('PlatformTHE', 'NNP'), ('DEMOCRATIC', 'NNP'), ('PARTY', 'NNP'), ('HOLDS', 'NNP'), ('TO', 'NNP'), ('THESE', 'NNP'), ('PRINCIPLES', 'NNP'), (':', ':'), ('That', 'DT'), ('government', 'NN'), ('functions', 'NNS'), ('best', 'RB'), ('when', 'WRB'), ('it', 'PRP'), ('is', 'VBZ'), ('closest', 'VBN'), ('to', 'TO'), ('the', 'DT'), ('people.That', 'NN'), ('Democrats', 'NNPS'), ('are', 'VBP'), ('determined', 'VBN'), ('to', 'TO'), ('provide', 'VB'), ('economic', 'JJ'), ('opportunity', 'NN'), ('for', 'IN'), ('all', 'DT'), ('Alabamians', 'NNPS'), ('.', '.')], [('It', 'PRP'), ('is', 'VBZ'), ('our', 'PRP$'), ('purpose', 'JJ'), ('to', 'TO'), ('take', 'VB'), ('maximum', 'JJ'), ('advantage', 'NN'), ('of', 'IN'), ('the', 'DT'), ('bountiful', 'JJ'), ('natural', 'JJ'), ('resources', 'NNS'), ('with', 'IN'), ('which', 'WDT'), ('we', 'PRP'), ('are', 'VBP'), ('blessed', 'VBN'), ('.', '.')], [('We', 'PRP'), ('work', 'VBP'), ('for', 'IN'), ('the', 'DT

In [9]:
#creating functions to filter out parts of speech
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
#running pos filter
d['tokens_sentences_lemmatized'] = d['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

# Removing stopwords

In [11]:
from nltk.corpus import stopwords
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make','party', 'see', 'want', 'come', 'take', 'use', 'would', 'can','bjp','congress','ensure','provide','state','government','establish','give','development','support','believe','must','right']
stopwords_other = ['one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something','shall','maharashtra','karnataka','meghalaya','bihar','assam','haryana','odisha','delhi','implement','develop','policy','new','nagaland','punjab']
my_stopwords = stopwords.words('english') + stopwords_verbs + stopwords_other

In [12]:
from itertools import chain
d['tokens'] = d['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
d['tokens'] = d['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

## Parsing only nouns to get only meaningful words 

In [13]:
for i in range(len(d)):
    x=''
    for tokens in d['tokens'][i]:
        x+=tokens
        x+=' '
    d['tokens'][i] = []
    for word,pos in nltk.pos_tag(nltk.word_tokenize(str(x))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            d['tokens'][i].append(word)

In [14]:
d['tokens']

0      [platformthe, principles, function, closest, d...
1      [platform, represent, alaska, group, pledge, w...
2      [gop, platform, mission, statementthe, mission...
3      [platformarizona, democrats, tomorrowresponsib...
4      [platformwe, difference, intend, difference, p...
                             ...                        
322    [wisconsin, platform, wisconsin, platform, cam...
323    [resolutions, resolutions, education, supports...
324    [platform, convention, june, charlotte, repres...
325    [wisconsin, resolutions, resolutions, conventi...
326    [convention, resolutions, resolutions, convent...
Name: tokens, Length: 327, dtype: object

In [38]:
from gensim.models import Phrases
tokens = d['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

C:\Users\HP\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [39]:
from gensim import corpora
dictionary_LDA = corpora.Dictionary(tokens)

corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

In [40]:
from gensim import models
import numpy as np

### Making LDA model

In [43]:
np.random.seed(123456)
num_topics = 15
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

Wall time: 12.6 s


# Topics and their respective words

In [44]:
x = []
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=30):
    print(str(i)+": "+ topic)
    x.append(str(i)+": "+ topic)
    print()

0: 0.012*"convention" + 0.009*"rule" + 0.009*"education" + 0.007*"program" + 0.006*"community" + 0.006*"system" + 0.005*"people" + 0.005*"school" + 0.005*"child" + 0.005*"service" + 0.005*"business" + 0.005*"law" + 0.005*"democrats" + 0.004*"texas" + 0.004*"security" + 0.004*"effort" + 0.004*"district" + 0.004*"nation" + 0.004*"number" + 0.004*"family" + 0.004*"work" + 0.004*"level" + 0.003*"person" + 0.003*"states" + 0.003*"opportunity" + 0.003*"member" + 0.003*"time" + 0.003*"election" + 0.003*"health_care" + 0.003*"resource"

1: 0.017*"education" + 0.011*"virginia" + 0.011*"effort" + 0.011*"work" + 0.011*"family" + 0.010*"people" + 0.009*"opportunity" + 0.009*"job" + 0.009*"business" + 0.009*"health_care" + 0.008*"program" + 0.008*"community" + 0.008*"child" + 0.008*"economy" + 0.008*"school" + 0.007*"service" + 0.007*"system" + 0.007*"citizen" + 0.006*"law" + 0.006*"environment" + 0.006*"vermont" + 0.005*"democrats" + 0.005*"responsibility" + 0.005*"access" + 0.005*"life" + 0.005*"

## Calculating the scores for each manifesto 

In [45]:
document = d['Manifesto'][45]
tokens = word_tokenize(document)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)

x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])

In [46]:
x

,topic #,weight,words in topic
0,2,0.07,"0.024*""convention"" + 0.023*""committee"" + 0.017..."
1,3,0.29,"0.014*""education"" + 0.013*""program"" + 0.010*""l..."
2,5,0.20,"0.012*""education"" + 0.011*""people"" + 0.011*""la..."
3,13,0.44,"0.010*""people"" + 0.010*""education"" + 0.010*""sc..."


In [47]:
#An example of how we can find the score of each topic along with the weights
name = 'family'
weight = 0
for i in range(len(x)):
    print(i)
    print(weight)
    topics_word = x['words in topic'][i].split('+')
    for j in range(len(topics_word)):
        index = topics_word[j].find(name)
        if index!=-1:
            weight+= float(topics_word[j].split('*')[0].split('.')[1])

0
0
1
0
2
10.0
3
18.0


In [48]:
#creating the list of variables & topa = topic_allocation
topa = [['education','school','teacher'],['work','business'],['job','employee','employer'],['economy'],['tax'],['security','protection','safety','crime'],['law'],['health','health_care'],['environment','water','air','waste'],['constitution','rights','right'],['gas','oil','resources','resource'],['rural','poor']]

## Topic allocation code

In [50]:
Education = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=0
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Education.append(weight)


0
0
1
1.8699999898672104
2
8.310000106692314
3
9.010000109672546
4
10.450000077486038
5
11.220000080764294
0
12.920000098645687
1
13.800000093877316
2
17.020000152289867
3
17.650000154972076
4
18.610000133514404
5
19.100000135600567
0
20.80000015348196
1
20.80000015348196
2
20.80000015348196
3
20.80000015348196
4
20.80000015348196
5
20.80000015348196
0
0
1
0.11999999731779099
2
7.820000164210796
3
8.020000159740448
4
9.100000202655792
5
9.320000197738409
0
12.220000114291906
1
12.220000114291906
2
16.07000019773841
3
16.250000193715096
4
16.970000222325325
5
17.170000217854977
0
20.070000134408474
1
20.070000134408474
2
20.070000134408474
3
20.070000134408474
4
20.070000134408474
5
20.070000134408474
0
0
1
0.1599999964237213
2
3.940000146627426
3
8.260000318288803
4
8.390000315383077
0
11.590000243857503
1
11.590000243857503
2
13.480000318959355
3
16.360000433400273
4
16.400000432506204
0
19.60000036098063
1
19.60000036098063
2
19.60000036098063
3
19.60000036098063
4
19.60000036098063


0
0
1
0.3399999924004078
2
0.41999999061226845
3
4.200000140815973
4
5.640000108629465
0
11.43999994173646
1
11.599999938160181
2
11.599999938160181
3
13.490000013262033
4
14.449999991804361
0
20.249999824911356
1
20.249999824911356
2
20.249999824911356
3
20.249999824911356
4
20.249999824911356
0
0
1
0.1599999964237213
2
5.200000196695328
3
7.480000168085098
0
11.480000227689743
1
11.480000227689743
2
14.000000327825546
3
15.52000030875206
0
19.520000368356705
1
19.520000368356705
2
19.520000368356705
3
19.520000368356705
0
0
1
1.0199999772012234
2
7.040000077337027
3
7.7600000612437725
4
8.930000107735395
0
12.530000250786543
1
13.010000240057707
2
16.02000029012561
3
16.500000279396772
4
16.860000293701887
0
20.460000436753035
1
20.460000436753035
2
20.460000436753035
3
20.460000436753035
4
20.460000436753035
0
0
1
0.1599999964237213
2
4.360000163316727
3
5.460000157356262
4
7.860000193119049
5
8.25000018440187
0
11.550000315532088
1
11.550000315532088
2
13.650000398978591
3
14.64000

0
0
1
0.1599999964237213
2
7.579999595880508
3
7.679999593645334
4
9.239999536424875
0
11.939999643713236
1
11.939999643713236
2
15.64999944344163
3
15.739999441429973
4
16.779999403283
0
19.47999951057136
1
19.47999951057136
2
19.47999951057136
3
19.47999951057136
4
19.47999951057136
0
0
1
2.0399999544024467
2
2.1599999517202377
3
3.6999999433755875
4
8.979999914765358
5
9.239999908953905
0
12.039999920874834
1
12.999999899417162
2
12.999999899417162
3
13.769999895244837
4
17.28999987617135
5
17.36999987438321
0
20.16999988630414
1
20.16999988630414
2
20.16999988630414
3
20.16999988630414
4
20.16999988630414
5
20.16999988630414
0
0
1
0.850000012665987
2
1.0100000090897083
3
7.0300001092255116
4
7.230000104755163
5
9.390000190585852
0
12.090000297874212
1
12.490000303834677
2
12.490000303834677
3
15.500000353902578
4
15.680000349879265
5
17.120000407099724
0
19.820000514388084
1
19.820000514388084
2
19.820000514388084
3
19.820000514388084
4
19.820000514388084
5
19.820000514388084
0
0
1

0
0
1
0.2800000011920929
2
3.780000001192093
3
5.079999953508377
4
8.439999967813492
5
8.569999964907765
0
11.270000072196126
1
11.270000072196126
2
13.020000072196126
3
14.190000029280782
4
16.430000038817525
5
16.470000037923455
0
19.170000145211816
1
19.170000145211816
2
19.170000145211816
3
19.170000145211816
4
19.170000145211816
5
19.170000145211816
0
0
1
0.4000000059604645
2
3.3399999141693115
3
7.539999842643738
0
10.839999973773956
1
10.839999973773956
2
12.30999992787838
3
15.109999880194664
0
18.410000011324883
1
18.410000011324883
2
18.410000011324883
3
18.410000011324883
0
0
1
0.11999999731779099
2
5.720000080764294
3
9.19999998062849
0
11.99999999254942
1
11.99999999254942
2
14.80000003427267
3
17.11999996751547
0
19.919999979436398
1
19.919999979436398
2
19.919999979436398
3
19.919999979436398
0
0
1
0.3399999924004078
2
0.4999999888241291
3
6.940000105649233
4
8.740000177174807
5
9.2600001655519
0
12.26000028476119
1
12.420000281184912
2
12.420000281184912
3
15.6400003395

0
0
1
0.5600000023841858
2
2.6600000858306885
3
6.260000228881836
0
10.26000028848648
1
10.26000028848648
2
11.310000330209732
3
13.710000425577164
0
17.71000048518181
1
17.71000048518181
2
17.71000048518181
3
17.71000048518181
0
0
1
0.1699999962002039
2
0.3299999926239252
3
3.1300000343471766
4
3.53000002540648
5
4.1300000343471766
6
4.650000022724271
0
10.850000070407987
1
10.930000068619847
2
10.930000068619847
3
12.330000089481473
4
12.690000081434846
5
13.09000008739531
6
13.250000083819032
0
19.450000131502748
1
19.450000131502748
2
19.450000131502748
3
19.450000131502748
4
19.450000131502748
5
19.450000131502748
6
19.450000131502748
0
0
1
0.3399999924004078
2
0.53999999538064
3
8.659999761730433
4
9.059999752789736
5
9.659999761730433
0
12.159999761730433
1
12.319999758154154
2
12.319999758154154
3
16.37999964132905
4
16.739999633282423
5
17.139999639242887
0
19.639999639242887
1
19.639999639242887
2
19.639999639242887
3
19.639999639242887
4
19.639999639242887
5
19.6399996392428

...

0
0
1
0.3199999928474426
2
3.5400000512599945
3
6.900000065565109
0
10.799999922513962
1
10.799999922513962
2
12.409999951720238
3
14.649999961256981
0
18.549999818205833
1
18.549999818205833
2
18.549999818205833
3
18.549999818205833
0
0
1
0.3199999928474426
2
3.5400000512599945
3
6.900000065565109
0
10.799999922513962
1
10.799999922513962
2
12.409999951720238
3
14.649999961256981
0
18.549999818205833
1
18.549999818205833
2
18.549999818205833
3
18.549999818205833
0
0
1
5.879999816417694
2
6.1799998097121716
3
7.139999788254499
4
8.829999726265669
0
12.129999857395887
1
15.069999765604734
2
15.339999759569764
3
15.97999974526465
4
16.499999726191163
0
19.79999985732138
1
19.79999985732138
2
19.79999985732138
3
19.79999985732138
4
19.79999985732138
0
0
1
0.2800000011920929
2
6.5799998342990875
3
7.27999983727932
4
8.239999815821648
5
8.369999812915921
0
11.46999983675778
1
11.46999983675778
2
14.619999753311276
3
15.249999755993485
4
15.88999974168837
5
15.929999740794301
0
19.0299997646

1
0.23999999463558197
2
5.55999992787838
3
8.679999813437462
4
8.809999810531735
0
11.709999727085233
1
11.709999727085233
2
14.369999693706632
3
16.449999617412686
4
16.489999616518617
0
19.389999533072114
1
19.389999533072114
2
19.389999533072114
3
19.389999533072114
4
19.389999533072114
0
0
1
0.03999999910593033
2
7.319999732077122
3
7.519999727606773
4
8.959999695420265
0
12.159999623894691
1
12.159999623894691
2
15.799999490380287
3
15.979999486356974
4
16.9399994648993
0
20.139999393373728
1
20.139999393373728
2
20.139999393373728
3
20.139999393373728
4
20.139999393373728
0
0
1
0.1599999964237213
2
4.639999896287918
3
8.47999981045723
4
8.739999804645777
0
11.739999923855066
1
11.739999923855066
2
13.979999873787165
3
16.539999816566706
4
16.619999814778566
0
19.619999933987856
1
19.619999933987856
2
19.619999933987856
3
19.619999933987856
4
19.619999933987856
0
0
1
0.5099999886006117
2
0.7899999897927046
3
5.689999906346202
4
8.809999791905284
0
11.709999708458781
1
11.949999703

0
0
1
0.23999999463558197
2
7.659999594092369
3
7.759999591857195
4
9.079999584704638
0
11.979999501258135
1
11.979999501258135
2
15.689999300986528
3
15.779999298974872
4
16.6599992942065
0
19.559999210759997
1
19.559999210759997
2
19.559999210759997
3
19.559999210759997
4
19.559999210759997
0
0
1
0.23999999463558197
2
5.55999992787838
3
5.7599999234080315
4
7.92000000923872
0
11.320000045001507
1
11.320000045001507
2
13.980000011622906
3
14.160000007599592
4
15.600000064820051
0
19.000000100582838
1
19.000000100582838
2
19.000000100582838
3
19.000000100582838
4
19.000000100582838
0
0
1
0.2800000011920929
2
6.720000118017197
3
7.020000111311674
4
8.460000079125166
5
8.570000076666474
0
11.570000195875764
1
11.570000195875764
2
14.790000254288316
3
15.060000248253345
4
16.020000226795673
5
16.1200002245605
0
19.12000034376979
1
19.12000034376979
2
19.12000034376979
3
19.12000034376979
4
19.12000034376979
5
19.12000034376979
0
0
1
0.20000000298023224
2
3.2799999862909317
3
5.19999994337

0
0
1
1.0199999772012234
2
1.1799999736249447
3
7.339999940246344
4
8.659999933093786
0
11.960000064224005
1
12.440000053495169
2
12.440000053495169
3
15.520000036805868
4
16.400000032037497
0
19.700000163167715
1
19.700000163167715
2
19.700000163167715
3
19.700000163167715
4
19.700000163167715
0
0
1
0.1699999962002039
2
0.2899999935179949
3
7.709999592974782
4
7.909999588504434
5
9.349999556317925
0
12.149999568238854
1
12.229999566450715
2
12.229999566450715
3
15.939999366179109
4
16.119999362155795
5
17.079999340698123
0
19.879999352619052
1
19.879999352619052
2
19.879999352619052
3
19.879999352619052
4
19.879999352619052
5
19.879999352619052
0
0
1
0.20000000298023224
2
3.839999869465828
3
4.03999986499548
4
7.03999986499548
5
7.169999862089753
0
11.169999921694398
1
11.169999921694398
2
12.989999854937196
3
13.169999850913882
4
15.169999850913882
5
15.209999850019813
0
19.209999909624457
1
19.209999909624457
2
19.209999909624457
3
19.209999909624457
4
19.209999909624457
5
19.209999

0
0
1
0.5099999886006117
2
0.7099999915808439
3
4.6300000082701445
4
7.6300000082701445
5
7.760000005364418
0
11.559999957680702
1
11.799999952316284
2
11.799999952316284
3
13.759999960660934
4
15.759999960660934
5
15.799999959766865
0
19.59999991208315
1
19.59999991208315
2
19.59999991208315
3
19.59999991208315
4
19.59999991208315
5
19.59999991208315
0
0
1
0.1599999964237213
2
7.020000129938126
3
8.460000097751617
0
12.060000240802765
1
12.060000240802765
2
15.490000307559967
3
16.450000286102295
0
20.050000429153442
1
20.050000429153442
2
20.050000429153442
3
20.050000429153442
0
0
1
1.5300000607967377
2
6.150000244379044
3
9.510000258684158
0
12.410000175237656
1
13.130000203847885
2
15.440000295639038
3
17.68000030517578
0
20.58000022172928
1
20.58000022172928
2
20.58000022172928
3
20.58000022172928
0
0
1
0.23999999463558197
2
2.6200000196695328
3
6.580000177025795
0
10.980000153183937
1
10.980000153183937
2
12.170000165700912
3
14.810000270605087
0
19.21000024676323
1
19.210000246

### For Business

In [51]:
Business = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=1
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Business.append(weight)


0
0
1
1.209999993443489
2
6.270000085234642
3
6.550000086426735
4
7.390000067651272
5
7.810000069439411
0
10.020000092685223
1
11.010000087320805
2
13.310000129044056
3
13.730000130832195
4
14.21000012010336
5
14.490000121295452
0
0
1
0
2
6.0500001311302185
3
6.130000129342079
4
6.76000015437603
5
7.060000147670507
0
10.830000039190054
1
10.980000035837293
2
13.730000095441937
3
13.850000092759728
4
14.210000107064843
5
14.330000104382634
0
0
1
0
2
2.9700001180171967
3
5.490000218153
4
5.6200002152472734
0
9.780000122264028
1
9.98000011779368
2
11.33000017143786
3
12.770000228658319
4
12.840000227093697
0
0
1
0.6599999852478504
2
0.6599999852478504
3
1.9799999557435513
4
3.4499999098479748
5
3.7499999031424522
0
11.289999686181545
1
11.829999674111605
2
11.879999672994018
3
12.479999659582973
4
13.319999633356929
5
13.43999963067472
0
0
1
0.2199999950826168
2
0.2199999950826168
3
1.7600000016391277
4
3.999999951571226
0
9.98000006005168
1
10.160000056028366
2
10.410000059753656
3
11.11

0
0
1
0.1099999975413084
2
0.1099999975413084
3
1.9800000172108412
4
2.4700000192970037
0
11.569999864324927
1
11.65999986231327
2
11.80999985896051
3
12.659999867901206
4
12.939999869093299
0
0
1
0
2
4.400000065565109
3
4.5200000628829
4
5.570000104606152
5
5.700000101700425
0
9.99000027216971
1
10.290000265464187
2
12.290000295266509
3
12.470000291243196
4
13.070000315085053
5
13.140000313520432
0
0
1
0
2
3.519999921321869
3
3.5999999195337296
4
5.700000002980232
0
9.730000033974648
1
9.9300000295043
2
11.529999993741512
3
11.649999991059303
4
12.850000038743019
0
0
1
0
2
2.419999986886978
3
2.65999998152256
4
4.060000002384186
5
4.4499999936670065
6
4.749999986961484
0
10.2099998164922
1
10.45999982021749
2
11.559999814257026
3
11.919999806210399
4
12.719999818131328
5
12.929999813437462
6
13.049999810755253
0
0
1
1.980000078678131
2
3.8700001537799835
3
4.390000142157078
0
10.890000142157078
1
11.790000177919865
2
12.870000220835209
3
13.150000214576721
0
0
1
0
2
5.390000104904175


0
0
1
0
2
3.080000013113022
3
3.1600000113248825
4
4.629999965429306
5
4.75999996252358
0
10.350000055506825
1
10.500000052154064
2
11.900000058114529
3
12.02000005543232
4
12.860000029206276
5
12.930000027641654
0
0
1
0
2
0.2199999950826168
3
1.339999970048666
0
7.579999830573797
1
9.27999984845519
2
9.379999846220016
3
10.019999831914902
0
0
1
0
2
5.829999685287476
3
5.989999681711197
4
6.759999677538872
0
10.26999981701374
1
10.469999812543392
2
13.119999669492245
3
13.359999664127827
4
13.799999661743641
0
0
1
1.1000000163912773
2
1.1000000163912773
3
5.500000081956387
4
6.060000069439411
0
10.740000255405903
1
11.640000268816948
2
11.8400002643466
3
13.840000294148922
4
14.160000286996365
0
0
1
0.2199999950826168
2
0.2199999950826168
3
3.7399999164044857
4
5.139999937266111
0
10.730000030249357
1
10.910000026226044
2
11.01000002399087
3
12.609999988228083
4
13.410000000149012
0
0
1
0.8799999803304672
2
6.379999980330467
3
6.799999970942736
0
11.349999893456697
1
12.069999877363443

0
0
1
0
2
5.169999986886978
3
5.3299999833106995
4
5.889999970793724
0
10.700000032782555
1
10.850000029429793
2
13.200000023469329
3
13.44000001810491
4
13.760000010952353
0
0
1
0
2
3.3000001311302185
3
5.190000206232071
0
10.39000028371811
1
10.490000281482935
2
11.99000034108758
3
13.070000384002924
0
0
1
0.3299999926239252
2
0.3299999926239252
3
5.060000071302056
4
5.830000067129731
0
10.89999988116324
1
11.16999987512827
2
11.319999871775508
3
13.469999907538295
4
13.909999905154109
0
0
1
0.3299999926239252
2
0.3299999926239252
3
3.079999992623925
4
3.1199999917298555
5
5.359999941661954
0
10.299999879673123
1
10.569999873638153
2
10.669999871402979
3
11.919999871402979
4
11.979999870061874
5
13.259999841451645
0
0
1
1.209999993443489
2
5.94000007212162
3
6.780000053346157
0
11.20000009983778
1
12.190000094473362
2
14.340000130236149
3
14.820000119507313
0
0
1
0.7700000032782555
2
0.7700000032782555
3
4.070000134408474
4
4.700000159442425
0
11.200000159442425
1
11.830000162124634


0
0
1
0.3299999926239252
2
0.3299999926239252
3
1.8699999991804361
4
3.7600000742822886
0
10.519999826326966
1
10.789999820291996
2
10.989999815821648
3
11.68999981880188
4
12.769999861717224
0
0
1
0
2
5.829999685287476
3
6.069999679923058
4
6.699999704957008
0
10.469999596476555
1
10.619999593123794
2
13.269999450072646
3
13.629999442026019
4
13.989999456331134
0
0
1
0.2199999950826168
2
0.2199999950826168
3
2.639999981969595
4
4.040000002831221
5
4.169999999925494
0
10.669999999925494
1
10.84999999590218
2
11.049999991431832
3
12.149999985471368
4
12.949999997392297
5
13.019999995827675
0
0
1
0
2
4.5099999606609344
3
4.749999955296516
4
6.149999976158142
5
6.409999970346689
0
9.65999997034669
1
9.959999963641167
2
12.009999945759773
3
12.369999937713146
4
13.169999949634075
5
13.309999946504831
0
0
1
2.0899999737739563
2
4.539999932050705
0
10.520000040531158
1
11.47000002861023
2
12.870000004768372
0
0
1
0.8799999803304672
2
0.8799999803304672
3
5.05999992787838
4
5.479999918490648


0
0
1
0
2
5.71999979019165
3
5.799999788403511
4
6.639999769628048
0
10.799999676644802
1
10.849999675527215
2
13.449999580159783
3
13.569999577477574
4
14.049999566748738
0
0
1
0
2
3.519999921321869
3
5.759999871253967
4
6.019999865442514
0
9.92000002041459
1
10.120000015944242
2
11.719999980181456
3
12.999999951571226
4
13.139999948441982
0
0
1
0.3299999926239252
2
0.3299999926239252
3
4.179999927058816
4
5.999999860301614
0
9.76999975182116
1
10.03999974578619
2
10.389999747276306
3
12.139999717473984
4
13.179999679327011
0
0
1
0
2
4.070000052452087
3
4.190000049769878
4
4.960000045597553
0
10.160000123083591
1
10.460000116378069
2
12.310000140219927
3
12.490000136196613
4
12.930000133812428
0
0
1
0
2
4.400000065565109
3
4.760000079870224
4
5.880000054836273
0
10.170000225305557
1
10.22000022418797
2
12.220000253990293
3
12.760000275447965
4
13.40000026114285
0
0
1
0
2
3.3000001311302185
3
3.380000129342079
4
5.480000212788582
0
9.640000119805336
1
9.890000123530626
2
11.39000018313

0
0
1
0
2
3.4100000262260437
3
5.720000118017197
0
9.620000272989273
1
9.92000026628375
2
11.47000027820468
3
12.790000330656767
0
0
1
0.1099999975413084
2
0.1099999975413084
3
3.850000036880374
4
3.9300000350922346
5
4.91000003926456
6
5.040000036358833
0
10.240000113844872
1
10.330000111833215
2
10.730000102892518
3
12.430000120773911
4
12.550000118091702
5
13.110000120475888
6
13.180000118911266
0
0
1
3.3000001311302185
2
3.5400001257658005
3
4.310000121593475
4
4.96000013127923
0
11.199999991804361
1
12.700000051409006
2
13.060000043362379
3
13.500000040978193
4
13.8500000461936
0
0
1
0.1099999975413084
2
0.1099999975413084
3
3.7400001417845488
4
3.8600001391023397
5
4.840000143274665
6
5.100000137463212
0
10.16999995149672
1
10.259999949485064
2
10.659999940544367
3
12.310000006109476
4
12.490000002086163
5
13.050000004470348
6
13.190000001341105
0
0
1
3.3000001311302185
2
3.5400001257658005
3
4.310000121593475
4
4.96000013127923
0
11.199999991804361
1
12.700000051409006
2
13.0600

0
0
1
0
2
3.3000001311302185
3
3.380000129342079
4
5.480000212788582
0
9.770000383257866
1
9.970000378787518
2
11.470000438392162
3
11.590000435709953
4
12.79000048339367
0
0
1
0.2199999950826168
2
0.2199999950826168
3
1.7600000016391277
4
2.3900000266730785
0
11.489999871701002
1
11.669999867677689
2
11.86999986320734
3
12.569999866187572
4
12.929999880492687
0
0
1
1.980000078678131
2
3.8700001537799835
3
4.650000136345625
0
11.020000260323286
1
11.920000296086073
2
13.000000339001417
3
13.420000329613686
0
0
1
2.2000000327825546
2
3.250000074505806
0
11.699999764561653
1
12.699999779462814
2
13.299999803304672
0
0
1
0
2
4.400000065565109
3
4.560000061988831
4
6.030000016093254
0
9.93000017106533
1
10.230000164359808
2
12.23000019416213
3
12.470000188797712
4
13.310000162571669
0
0
1
0
2
1.3199999704957008
3
3.5599999204277992
0
10.449999548494816
1
10.599999545142055
2
11.19999953173101
3
12.47999950312078
0
0
1
0
2
1.980000078678131
3
2.0600000768899918
4
3.6700001060962677
0
10.429

## Jobs and Labour relation

In [52]:
Labour = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=2
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Labour.append(weight)

0
0
1
0.989999994635582
2
2.830000028014183
3
3.2500000298023224
4
3.7300000190734863
5
3.7300000190734863
0
4.750000029802322
1
4.750000029802322
2
4.750000029802322
3
4.750000029802322
4
4.750000029802322
5
4.750000029802322
0
4.750000029802322
1
4.750000029802322
2
4.750000029802322
3
4.750000029802322
4
4.750000029802322
5
4.750000029802322
0
0
1
0
2
2.200000047683716
3
2.320000045001507
4
2.6800000593066216
5
2.8000000566244125
0
4.540000006556511
1
4.540000006556511
2
4.540000006556511
3
4.540000006556511
4
4.540000006556511
5
4.540000006556511
0
4.540000006556511
1
4.540000006556511
2
4.540000006556511
3
4.540000006556511
4
4.540000006556511
5
4.540000006556511
0
0
1
0
2
1.0800000429153442
3
2.5200001001358032
4
2.5800000987946987
0
4.5000000558793545
1
4.5000000558793545
2
4.5000000558793545
3
4.5000000558793545
4
4.5000000558793545
0
4.5000000558793545
1
4.5000000558793545
2
4.5000000558793545
3
4.5000000558793545
4
4.5000000558793545
0
0
1
0.5399999879300594
2
0.5399999879300

0
0
1
2.8399999141693115
2
3.1999999061226845
0
4.119999922811985
1
4.119999922811985
2
4.119999922811985
0
4.119999922811985
1
4.119999922811985
2
4.119999922811985
0
0
1
0.17999999597668648
2
0.17999999597668648
3
1.2600000388920307
4
1.7400000281631947
0
5.219999928027391
1
5.219999928027391
2
5.219999928027391
3
5.219999928027391
4
5.219999928027391
0
5.219999928027391
1
5.219999928027391
2
5.219999928027391
3
5.219999928027391
4
5.219999928027391
0
0
1
0
2
1.440000057220459
3
2.200000047683716
0
4.600000083446503
1
4.600000083446503
2
4.600000083446503
3
4.600000083446503
0
4.600000083446503
1
4.600000083446503
2
4.600000083446503
3
4.600000083446503
0
0
1
0.5399999879300594
2
2.260000016540289
3
2.500000011175871
4
3.040000032633543
0
5.2000001184642315
1
5.2000001184642315
2
5.2000001184642315
3
5.2000001184642315
4
5.2000001184642315
0
5.2000001184642315
1
5.2000001184642315
2
5.2000001184642315
3
5.2000001184642315
4
5.2000001184642315
0
0
1
0
2
1.2000000476837158
3
1.86000004

0
0
1
1.0799999758601189
2
1.0799999758601189
3
1.519999973475933
4
3.27999996393919
5
3.399999961256981
0
5.079999968409538
1
5.079999968409538
2
5.079999968409538
3
5.079999968409538
4
5.079999968409538
5
5.079999968409538
0
5.079999968409538
1
5.079999968409538
2
5.079999968409538
3
5.079999968409538
4
5.079999968409538
5
5.079999968409538
0
0
1
0.45000000670552254
2
0.45000000670552254
3
2.170000035315752
4
2.290000032633543
5
3.0100000612437725
0
4.630000125616789
1
4.630000125616789
2
4.630000125616789
3
4.630000125616789
4
4.630000125616789
5
4.630000125616789
0
4.630000125616789
1
4.630000125616789
2
4.630000125616789
3
4.630000125616789
4
4.630000125616789
5
4.630000125616789
0
0
1
0.17999999597668648
2
0.17999999597668648
3
1.3399999625980854
4
1.399999961256981
5
2.1199999898672104
6
2.179999988526106
0
4.639999967068434
1
4.639999967068434
2
4.639999967068434
3
4.639999967068434
4
4.639999967068434
5
4.639999967068434
6
4.639999967068434
0
4.639999967068434
1
4.639999967068

0
0
1
0
2
1.1200000047683716
3
1.2400000020861626
4
2.079999975860119
5
2.1399999745190144
0
4.720000017434359
1
4.720000017434359
2
4.720000017434359
3
4.720000017434359
4
4.720000017434359
5
4.720000017434359
0
4.720000017434359
1
4.720000017434359
2
4.720000017434359
3
4.720000017434359
4
4.720000017434359
5
4.720000017434359
0
0
1
0
2
0.07999999821186066
3
0.7199999839067459
0
3.5999999195337296
1
3.5999999195337296
2
3.5999999195337296
3
3.5999999195337296
0
3.5999999195337296
1
3.5999999195337296
2
3.5999999195337296
3
3.5999999195337296
0
0
1
0
2
2.119999885559082
3
2.359999880194664
4
2.799999877810478
0
4.419999942183495
1
4.419999942183495
2
4.419999942183495
3
4.419999942183495
4
4.419999942183495
0
4.419999942183495
1
4.419999942183495
2
4.419999942183495
3
4.419999942183495
4
4.419999942183495
0
0
1
0.9000000134110451
2
0.9000000134110451
3
2.500000037252903
4
2.8200000301003456
0
4.980000115931034
1
4.980000115931034
2
4.980000115931034
3
4.980000115931034
4
4.98000011593

0
0
1
0
2
0.9599999785423279
3
2.35999995470047
0
4.5200000405311584
1
4.5200000405311584
2
4.5200000405311584
3
4.5200000405311584
0
4.5200000405311584
1
4.5200000405311584
2
4.5200000405311584
3
4.5200000405311584
0
0
1
0.9200000166893005
2
1.9599999785423279
3
2.1399999745190144
0
4.900000024586916
1
4.900000024586916
2
4.900000024586916
3
4.900000024586916
0
4.900000024586916
1
4.900000024586916
2
4.900000024586916
3
4.900000024586916
0
0
1
0
2
1.0399999618530273
3
1.5999999642372131
0
4.119999885559082
1
4.119999885559082
2
4.119999885559082
3
4.119999885559082
0
4.119999885559082
1
4.119999885559082
2
4.119999885559082
3
4.119999885559082
0
0
1
0
2
0.47999998927116394
3
2.039999932050705
4
2.0999999307096004
0
4.619999852031469
1
4.619999852031469
2
4.619999852031469
3
4.619999852031469
4
4.619999852031469
0
4.619999852031469
1
4.619999852031469
2
4.619999852031469
3
4.619999852031469
4
4.619999852031469
0
0
1
0.2699999939650297
2
0.2699999939650297
3
1.3899999987334013
4
1.66999

...

0
0
1
0.2699999939650297
2
0.2699999939650297
3
0.8299999963492155
4
1.9100000392645597
0
5.029999924823642
1
5.029999924823642
2
5.029999924823642
3
5.029999924823642
4
5.029999924823642
0
5.029999924823642
1
5.029999924823642
2
5.029999924823642
3
5.029999924823642
4
5.029999924823642
0
0
1
0
2
2.119999885559082
3
2.479999877512455
4
2.8399998918175697
0
4.579999841749668
1
4.579999841749668
2
4.579999841749668
3
4.579999841749668
4
4.579999841749668
0
4.579999841749668
1
4.579999841749668
2
4.579999841749668
3
4.579999841749668
4
4.579999841749668
0
0
1
0.17999999597668648
2
0.17999999597668648
3
1.059999991208315
4
1.8600000031292439
5
1.9200000017881393
0
4.920000001788139
1
4.920000001788139
2
4.920000001788139
3
4.920000001788139
4
4.920000001788139
5
4.920000001788139
0
4.920000001788139
1
4.920000001788139
2
4.920000001788139
3
4.920000001788139
4
4.920000001788139
5
4.920000001788139
0
0
1
0
2
1.6399999856948853
3
1.9999999776482582
4
2.799999989569187
5
2.919999986886978
0
4

0
0
1
0
2
1.7999999523162842
3
2.039999946951866
4
2.399999961256981
0
4.61999998986721
1
4.61999998986721
2
4.61999998986721
3
4.61999998986721
4
4.61999998986721
0
4.61999998986721
1
4.61999998986721
2
4.61999998986721
3
4.61999998986721
4
4.61999998986721
0
0
1
0
2
1.159999966621399
3
2.240000009536743
0
4.699999988079071
1
4.699999988079071
2
4.699999988079071
3
4.699999988079071
0
4.699999988079071
1
4.699999988079071
2
4.699999988079071
3
4.699999988079071
0
0
1
0
2
2.5999999046325684
3
2.779999900609255
4
2.939999897032976
0
4.319999922066927
1
4.319999922066927
2
4.319999922066927
3
4.319999922066927
4
4.319999922066927
0
4.319999922066927
1
4.319999922066927
2
4.319999922066927
3
4.319999922066927
4
4.319999922066927
0
0
1
0
2
0.7200000286102295
3
0.780000027269125
4
1.1400000415742397
0
5.160000141710043
1
5.160000141710043
2
5.160000141710043
3
5.160000141710043
4
5.160000141710043
0
5.160000141710043
1
5.160000141710043
2
5.160000141710043
3
5.160000141710043
4
5.1600001417

0
0
1
0
2
1.7599999904632568
3
1.8199999891221523
4
2.379999991506338
0
4.299999948590994
1
4.299999948590994
2
4.299999948590994
3
4.299999948590994
4
4.299999948590994
0
4.299999948590994
1
4.299999948590994
2
4.299999948590994
3
4.299999948590994
4
4.299999948590994
0
0
1
0
2
1.399999976158142
3
2.559999942779541
4
2.679999940097332
0
4.359999947249889
1
4.359999947249889
2
4.359999947249889
3
4.359999947249889
4
4.359999947249889
0
4.359999947249889
1
4.359999947249889
2
4.359999947249889
3
4.359999947249889
4
4.359999947249889
0
0
1
0
2
1.4800000190734863
3
1.7200000137090683
4
2.2000000029802322
0
4.600000038743019
1
4.600000038743019
2
4.600000038743019
3
4.600000038743019
4
4.600000038743019
0
4.600000038743019
1
4.600000038743019
2
4.600000038743019
3
4.600000038743019
4
4.600000038743019
0
0
1
0.630000002682209
2
0.630000002682209
3
1.7500000074505806
4
1.9500000104308128
0
5.489999853074551
1
5.489999853074551
2
5.489999853074551
3
5.489999853074551
4
5.489999853074551
0
5.4

0
0
1
0
2
1.2400000095367432
3
2.5600000619888306
0
4.360000133514404
1
4.360000133514404
2
4.360000133514404
3
4.360000133514404
0
4.360000133514404
1
4.360000133514404
2
4.360000133514404
3
4.360000133514404
0
0
1
0.08999999798834324
2
0.08999999798834324
3
1.450000012293458
4
1.570000009611249
5
2.1300000119954348
6
2.1900000106543303
0
4.590000046417117
1
4.590000046417117
2
4.590000046417117
3
4.590000046417117
4
4.590000046417117
5
4.590000046417117
6
4.590000046417117
0
4.590000046417117
1
4.590000046417117
2
4.590000046417117
3
4.590000046417117
4
4.590000046417117
5
4.590000046417117
6
4.590000046417117
0
0
1
1.2000000476837158
2
1.5600000396370888
3
2.000000037252903
4
2.3000000417232513
0
5.179999977350235
1
5.179999977350235
2
5.179999977350235
3
5.179999977350235
4
5.179999977350235
0
5.179999977350235
1
5.179999977350235
2
5.179999977350235
3
5.179999977350235
4
5.179999977350235
0
0
1
0.08999999798834324
2
0.08999999798834324
3
1.4100000504404306
4
1.5900000464171171
5
2

0
0
1
0
2
1.0
3
1.8399999737739563
4
1.8999999724328518
0
4.599999900907278
1
4.599999900907278
2
4.599999900907278
3
4.599999900907278
4
4.599999900907278
0
4.599999900907278
1
4.599999900907278
2
4.599999900907278
3
4.599999900907278
4
4.599999900907278
0
0
1
0
2
2.0799999237060547
3
2.479999929666519
0
4.519999951124191
1
4.519999951124191
2
4.519999951124191
3
4.519999951124191
0
4.519999951124191
1
4.519999951124191
2
4.519999951124191
3
4.519999951124191
0
0
1
0
2
0.23999999463558197
3
0.6400000005960464
4
0.7599999979138374
0
2.380000062286854
1
2.380000062286854
2
2.380000062286854
3
2.380000062286854
4
2.380000062286854
0
2.380000062286854
1
2.380000062286854
2
2.380000062286854
3
2.380000062286854
4
2.380000062286854
0
0
1
0.45000000670552254
2
0.45000000670552254
3
2.129999954253435
4
2.5299999602138996
0
4.80999993160367
1
4.80999993160367
2
4.80999993160367
3
4.80999993160367
4
4.80999993160367
0
4.80999993160367
1
4.80999993160367
2
4.80999993160367
3
4.80999993160367
4
4

### Economy

In [53]:
Economy = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=3
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Economy.append(weight)


0
0
1
0.8799999952316284
2
2.7200000286102295
3
3.0000000298023224
4
3.0000000298023224
5
3.0000000298023224
0
0
1
0
2
2.200000047683716
3
2.2800000458955765
4
2.2800000458955765
5
2.360000044107437
0
0
1
0
2
1.0800000429153442
3
1.0800000429153442
4
1.1300000417977571
0
0
1
0.47999998927116394
2
0.47999998927116394
3
0.9599999785423279
4
0.9599999785423279
5
1.0399999767541885
0
0
1
0.1599999964237213
2
0.1599999964237213
3
0.7199999988079071
4
0.7199999988079071
0
0
1
0.23999999463558197
2
0.23999999463558197
3
1.3599999994039536
4
1.3599999994039536
5
1.4599999971687794
0
0
1
0
2
1.1200000047683716
3
1.1200000047683716
0
0
1
0.23999999463558197
2
0.23999999463558197
3
2.2000000327825546
4
2.2800000309944153
5
2.2800000309944153
0
0
1
0
2
0.9200000166893005
3
0.9200000166893005
0
0
1
0
2
1.9600000381469727
3
2.0400000363588333
4
2.0400000363588333
0
0
1
0.7200000286102295
2
2.3200000524520874
3
2.3200000524520874
4
2.4200000502169132
0
0
1
0.5600000023841858
2
0.5600000023841858
3
1.

0
0
1
0
2
0.4399999976158142
3
0.4399999976158142
0
0
1
0.23999999463558197
2
0.23999999463558197
3
1.7200000137090683
4
1.7200000137090683
0
0
1
2.640000104904175
2
2.760000102221966
3
2.760000102221966
0
0
1
0
2
1.5199999809265137
3
1.5199999809265137
0
0
1
0
2
0.20000000298023224
3
0.20000000298023224
0
0
1
0
2
1.2000000476837158
3
1.2000000476837158
0
0
1
0
2
2.200000047683716
3
2.360000044107437
4
2.360000044107437
5
2.41000004298985
0
0
1
0
2
1.600000023841858
3
1.600000023841858
0
0
1
0.7200000286102295
2
2.1600000858306885
3
2.1600000858306885
0
0
1
0.07999999821186066
2
2.359999969601631
3
2.439999967813492
4
2.439999967813492
5
2.599999964237213
0
0
1
0
2
0.8399999737739563
3
0.9599999710917473
4
0.9599999710917473
0
0
1
0.1599999964237213
2
0.1599999964237213
3
1.1999999582767487
4
1.1999999582767487
5
1.879999965429306
6
2.0799999609589577
0
0
1
0
2
0.6399999856948853
3
0.6399999856948853
0
0
1
0
2
0.6000000238418579
3
0.6000000238418579
0
0
1
0.07999999821186066
2
0.079999

0
0
1
0
2
1.840000033378601
3
1.8800000324845314
4
1.8800000324845314
0
0
1
0
2
0
0
0
1
0
2
1.5199999809265137
3
1.5199999809265137
4
1.5699999798089266
0
0
1
0
2
1.399999976158142
3
1.519999973475933
4
1.519999973475933
5
1.619999971240759
0
0
1
0
2
1.0
3
1.0
0
0
1
0
2
1.9199999570846558
3
1.9999999552965164
4
1.9999999552965164
5
2.1199999526143074
0
0
1
0
2
1.2000000476837158
3
1.2000000476837158
4
1.3000000454485416
5
1.340000044554472
0
0
1
0
2
1.7200000286102295
3
1.7200000286102295
4
1.7700000274926424
0
0
1
0
2
1.7999999523162842
3
1.9599999487400055
4
1.9599999487400055
0
0
1
0
2
1.159999966621399
3
1.159999966621399
0
0
1
0
2
2.5999999046325684
3
2.7199999019503593
4
2.7199999019503593
0
0
1
0
2
0.7200000286102295
3
0.7600000277161598
4
0.7600000277161598
0
0
1
0
2
1.6399999856948853
3
1.8799999803304672
4
1.8799999803304672
5
1.9299999792128801
0
0
1
0
2
1.399999976158142
3
1.399999976158142
4
1.499999973922968
0
0
1
0
2
1.399999976158142
3
1.399999976158142
4
1.499999973922

0
0
1
0
2
1.2799999713897705
3
1.7199999690055847
4
1.7199999690055847
5
1.8699999656528234
0
0
1
1.9600000381469727
2
1.9600000381469727
3
2.210000041872263
0
0
1
0.47999998927116394
2
0.47999998927116394
3
1.9600000083446503
4
1.9600000083446503
0
0
1
0.23999999463558197
2
0.23999999463558197
3
1.3599999994039536
4
1.3599999994039536
5
1.4099999982863665
0
0
1
0
2
1.9600000381469727
3
1.9600000381469727
0
0
1
0.7200000286102295
2
2.040000081062317
3
2.040000081062317
0
0
1
0
2
0.6800000071525574
3
0.6800000071525574
0
0
1
0
2
1.399999976158142
3
1.4799999743700027
4
1.4799999743700027
5
1.5299999732524157
0
0
1
0
2
1.2000000476837158
3
1.2800000458955765
4
1.2800000458955765
0
0
1
0.1599999964237213
2
0.1599999964237213
3
0.7199999988079071
4
0.7199999988079071
0
0
1
0.7200000286102295
2
0.7200000286102295
3
1.020000021904707
0
0
1
0.800000011920929
2
0.800000011920929
0
0
1
0
2
1.600000023841858
3
1.7600000202655792
4
1.7600000202655792
0
0
1
0
2
0.47999998927116394
3
0.479999989271

### Tax

In [54]:
Tax = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=4
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Tax.append(weight)


0
0
1
0.4399999976158142
2
2.2800000309944153
3
2.6300000324845314
4
3.4700000137090683
5
3.750000014901161
0
0
1
0
2
2.200000047683716
3
2.3000000454485416
4
2.9300000704824924
5
2.9300000704824924
0
0
1
0
2
1.0800000429153442
3
3.6000001430511475
4
3.6900001410394907
0
0
1
0.23999999463558197
2
0.23999999463558197
3
0.7199999839067459
4
2.1899999380111694
5
2.1899999380111694
0
0
1
0.07999999821186066
2
0.07999999821186066
3
0.6400000005960464
4
2.879999950528145
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.2400000020861626
4
2.0799999833106995
5
2.259999979287386
0
0
1
0
2
1.1200000047683716
3
2.310000017285347
0
0
1
0.11999999731779099
2
0.11999999731779099
3
2.0800000354647636
4
2.1800000332295895
5
2.8100000582635403
0
0
1
0
2
0.9200000166893005
3
3.369999974966049
0
0
1
0
2
1.9600000381469727
3
2.0600000359117985
4
2.9000000171363354
0
0
1
0.36000001430511475
2
1.9600000381469727
3
2.4500000402331352
4
2.6300000362098217
0
0
1
0.2800000011920929
2
0.2800000011920929
3
1.4

3
1.979999989271164
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.6000000163912773
4
2.0200000070035458
0
0
1
2.640000104904175
2
2.7900001015514135
3
3.350000089034438
0
0
1
0
2
1.5199999809265137
3
2.849999964237213
0
0
1
0
2
0.20000000298023224
3
0.40999999828636646
0
0
1
0
2
1.2000000476837158
3
3.439999997615814
0
0
1
0
2
2.200000047683716
3
2.4000000432133675
4
2.89000004529953
5
2.9800000432878733
0
0
1
0
2
1.600000023841858
3
4.6100000739097595
0
0
1
0.36000001430511475
2
1.8000000715255737
3
3.5500000715255737
0
0
1
0.03999999910593033
2
2.319999970495701
3
2.4199999682605267
4
3.0499999932944775
5
3.0499999932944775
0
0
1
0
2
0.8399999737739563
3
0.989999970421195
4
3.7900000121444464
0
0
1
0.07999999821186066
2
0.07999999821186066
3
1.119999960064888
4
2.099999964237213
5
3.2899999767541885
6
3.6499999687075615
0
0
1
0
2
0.6399999856948853
3
4.070000052452087
0
0
1
0
2
0.6000000238418579
3
2.7000001072883606
0
0
1
0.03999999910593033
2
0.03999999910593033
3
0.840000011

0
0
1
0
2
1.840000033378601
3
1.890000032261014
4
2.520000057294965
0
0
1
0.08999999798834324
2
0.08999999798834324
0
0
1
0
2
1.5199999809265137
3
3.479999989271164
4
3.569999987259507
0
0
1
0
2
1.399999976158142
3
1.5499999728053808
4
2.8799999561160803
5
3.0599999520927668
0
0
1
0
2
1.0
3
3.2399999499320984
0
0
1
0
2
1.9199999570846558
3
2.0199999548494816
4
2.9999999590218067
5
2.9999999590218067
0
0
1
0
2
1.2000000476837158
3
2.250000089406967
4
2.4300000853836536
5
2.4300000853836536
0
0
1
0
2
1.7200000286102295
3
3.050000011920929
4
3.140000009909272
0
0
1
0
2
1.7999999523162842
3
1.9999999478459358
4
2.6299999728798866
0
0
1
0
2
1.159999966621399
3
3.0500000417232513
0
0
1
0
2
2.5999999046325684
3
2.749999901279807
4
3.0299998950213194
0
0
1
0
2
0.7200000286102295
3
0.7700000274926424
4
1.4000000525265932
0
0
1
0
2
1.6399999856948853
3
1.9399999789893627
4
2.919999983161688
5
3.009999981150031
0
0
1
0
2
1.399999976158142
3
3.42999991774559
4
3.6099999137222767
0
0
1
0
2
1.399999

0
0
1
0
2
1.2799999713897705
3
1.8299999684095383
4
3.09000001847744
5
3.3600000124424696
0
0
1
1.9600000381469727
2
2.4500000402331352
3
2.9000000469386578
0
0
1
0.23999999463558197
2
0.23999999463558197
3
1.7200000137090683
4
3.260000005364418
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.2400000020861626
4
2.9900000020861626
5
3.080000000074506
0
0
1
0
2
1.9600000381469727
3
2.8000000193715096
0
0
1
0.36000001430511475
2
1.6800000667572021
3
3.6400000751018524
0
0
1
0
2
0.6800000071525574
3
2.9900000989437103
0
0
1
0
2
1.399999976158142
3
1.499999973922968
4
2.2699999697506428
5
2.359999967738986
0
0
1
0
2
1.2000000476837158
3
1.3000000454485416
4
3.4000001288950443
0
0
1
0.07999999821186066
2
0.07999999821186066
3
0.6400000005960464
4
1.2700000256299973
0
0
1
0.7200000286102295
2
2.610000103712082
3
3.1500000916421413
0
0
1
0.800000011920929
2
1.8500000536441803
0
0
1
0
2
1.600000023841858
3
1.8000000193715096
4
3.269999973475933
0
0
1
0
2
0.47999998927116394
3
2.719999939203

### Security

In [55]:
Security = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=5
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Security.append(weight)


0
0
1
0.4399999976158142
2
3.200000047683716
3
3.620000049471855
4
4.22000003606081
5
4.500000037252903
0
5.3500000461936
1
5.3500000461936
2
7.650000087916851
3
7.930000089108944
4
8.410000078380108
5
8.6900000795722
0
8.6900000795722
1
8.6900000795722
2
8.6900000795722
3
8.6900000795722
4
8.6900000795722
5
8.6900000795722
0
8.6900000795722
1
8.6900000795722
2
8.6900000795722
3
8.6900000795722
4
8.6900000795722
5
8.6900000795722
0
0
1
0
2
3.3000000715255737
3
3.4200000688433647
4
3.870000086724758
5
4.0300000831484795
0
5.480000041425228
1
5.480000041425228
2
8.230000101029873
3
8.310000099241734
4
8.670000113546848
5
8.750000111758709
0
8.750000111758709
1
8.750000111758709
2
8.750000111758709
3
8.750000111758709
4
8.750000111758709
5
8.850000109523535
0
8.850000109523535
1
8.850000109523535
2
8.850000109523535
3
8.850000109523535
4
8.850000109523535
5
8.850000109523535
0
0
1
0
2
1.6200000643730164
3
3.42000013589859
4
3.4800001345574856
0
5.080000098794699
1
5.080000098794699
2
6.43

0
0
1
0.11999999731779099
2
0.11999999731779099
3
2.3400000259280205
4
2.9400000125169754
5
4.079999998211861
0
5.379999950528145
1
5.379999950528145
2
5.379999950528145
3
7.229999974370003
4
7.709999963641167
5
8.659999951720238
0
8.659999951720238
1
8.659999951720238
2
8.659999951720238
3
8.659999951720238
4
8.659999951720238
5
8.659999951720238
0
8.659999951720238
1
8.659999951720238
2
8.659999951720238
3
8.659999951720238
4
8.659999951720238
5
8.659999951720238
0
0
1
0.1599999964237213
2
0.1599999964237213
3
1.1199999749660492
4
2.0699999630451202
5
2.1299999617040157
0
4.830000068992376
1
4.830000068992376
2
4.830000068992376
3
5.630000051110983
4
6.39000004157424
5
6.440000040456653
0
6.440000040456653
1
6.440000040456653
2
6.440000040456653
3
6.440000040456653
4
6.440000040456653
5
6.440000040456653
0
6.440000040456653
1
6.440000040456653
2
6.440000040456653
3
6.440000040456653
4
6.440000040456653
5
6.440000040456653
0
0
1
0.07999999821186066
2
0.07999999821186066
3
1.4600000232

0
0
1
0
2
1.8000000715255737
3
2.200000062584877
4
2.3800000585615635
0
4.979999963194132
1
4.979999963194132
2
6.480000022798777
3
6.800000015646219
4
6.950000012293458
0
6.950000012293458
1
6.950000012293458
2
6.950000012293458
3
6.950000012293458
4
6.950000012293458
0
6.950000012293458
1
6.950000012293458
2
6.950000012293458
3
6.950000012293458
4
6.950000012293458
0
0
1
0
2
1.7399999499320984
3
2.7399999648332596
0
4.939999952912331
1
4.939999952912331
2
6.389999911189079
3
7.189999923110008
0
7.189999923110008
1
7.189999923110008
2
7.189999923110008
3
7.189999923110008
0
7.189999923110008
1
7.189999923110008
2
7.189999923110008
3
7.189999923110008
0
0
1
0.03999999910593033
2
0.03999999910593033
3
1.0600000098347664
4
1.4100000113248825
0
4.909999951720238
1
4.909999951720238
2
4.909999951720238
3
5.7599999606609344
4
6.039999961853027
0
6.039999961853027
1
6.039999961853027
2
6.039999961853027
3
6.039999961853027
4
6.039999961853027
0
6.039999961853027
1
6.039999961853027
2
6.03999

0
0
1
0
2
3.2400001287460327
3
3.4800001233816147
4
3.9800001308321953
0
5.430000089108944
1
5.430000089108944
2
8.130000196397305
3
8.290000192821026
4
8.69000019878149
0
8.69000019878149
1
8.69000019878149
2
8.69000019878149
3
8.69000019878149
4
8.69000019878149
0
8.69000019878149
1
8.69000019878149
2
8.69000019878149
3
8.69000019878149
4
8.69000019878149
0
0
1
0
2
1.559999942779541
3
3.159999907016754
4
3.279999904334545
0
4.979999922215939
1
4.979999922215939
2
6.279999874532223
3
7.559999845921993
4
7.659999843686819
0
7.659999843686819
1
7.659999843686819
2
7.659999843686819
3
7.659999843686819
4
7.659999843686819
0
7.659999843686819
1
7.659999843686819
2
7.659999843686819
3
7.659999843686819
4
7.659999843686819
0
0
1
0
2
1.3199999928474426
3
2.670000046491623
4
2.7300000451505184
0
4.9300000332295895
1
4.9300000332295895
2
6.030000027269125
3
7.110000070184469
4
7.160000069066882
0
7.160000069066882
1
7.160000069066882
2
7.160000069066882
3
7.160000069066882
4
7.160000069066882


0
0
1
0
2
1.2599999606609344
3
3.009999930858612
0
4.659999996423721
1
4.659999996423721
2
5.709999963641167
3
7.109999939799309
0
7.109999939799309
1
7.109999939799309
2
7.109999939799309
3
7.109999939799309
0
7.109999939799309
1
7.109999939799309
2
7.109999939799309
3
7.109999939799309
0
0
1
0
2
2.400000035762787
3
3.8499999940395355
0
5.25
1
5.25
2
7.250000029802322
3
8.409999996423721
0
8.409999996423721
1
8.409999996423721
2
8.409999996423721
3
8.409999996423721
0
8.409999996423721
1
8.409999996423721
2
8.409999996423721
3
8.409999996423721
0
0
1
0.07999999821186066
2
0.07999999821186066
3
2.8400000482797623
4
3.5900000780820847
5
3.8300000727176666
0
5.330000132322311
1
5.330000132322311
2
5.330000132322311
3
7.630000174045563
4
8.23000019788742
5
8.430000193417072
0
8.430000193417072
1
8.430000193417072
2
8.430000193417072
3
8.430000193417072
4
8.430000193417072
5
8.430000193417072
0
8.430000193417072
1
8.430000193417072
2
8.430000193417072
3
8.430000193417072
4
8.43000019341707

...

0
0
1
0.20000000298023224
2
1.9399999529123306
3
3.290000006556511
0
5.189999982714653
1
5.189999982714653
2
6.639999940991402
3
7.719999983906746
0
7.719999983906746
1
7.719999983906746
2
7.719999983906746
3
7.719999983906746
0
7.719999983906746
1
7.719999983906746
2
7.719999983906746
3
7.719999983906746
0
0
1
0.07999999821186066
2
0.07999999821186066
3
2.30000002682209
4
2.420000024139881
5
2.7700000256299973
0
5.120000019669533
1
5.120000019669533
2
5.120000019669533
3
6.970000043511391
4
7.050000041723251
5
7.330000042915344
0
7.330000042915344
1
7.330000042915344
2
7.330000042915344
3
7.330000042915344
4
7.330000042915344
5
7.330000042915344
0
7.330000042915344
1
7.330000042915344
2
7.330000042915344
3
7.330000042915344
4
7.330000042915344
5
7.330000042915344
0
0
1
0
2
2.459999978542328
3
3.2999999821186066
4
3.75
5
3.869999997317791
0
5.2700000032782555
1
5.2700000032782555
2
7.319999985396862
3
7.879999987781048
4
8.240000002086163
5
8.339999999850988
0
8.339999999850988
1
8.339

0
0
1
0
2
2.2200000286102295
3
2.4600000232458115
4
3.0600000098347664
0
5.060000039637089
1
5.060000039637089
2
6.910000063478947
3
7.070000059902668
4
7.550000049173832
0
7.550000049173832
1
7.550000049173832
2
7.550000049173832
3
7.550000049173832
4
7.550000049173832
0
7.550000049173832
1
7.550000049173832
2
7.550000049173832
3
7.550000049173832
4
7.550000049173832
0
0
1
0.2800000011920929
2
0.2800000011920929
3
1.9600000083446503
4
2.2100000120699406
0
5.159999880939722
1
5.159999880939722
2
5.159999880939722
3
6.5599998869001865
4
6.759999889880419
0
6.759999889880419
1
6.759999889880419
2
6.759999889880419
3
6.759999889880419
4
6.759999889880419
0
6.759999889880419
1
6.759999889880419
2
6.759999889880419
3
6.759999889880419
4
6.759999889880419
0
0
1
0.03999999910593033
2
0.03999999910593033
3
1.840000070631504
4
1.960000067949295
5
2.510000064969063
0
4.760000005364418
1
4.760000005364418
2
4.760000005364418
3
6.260000064969063
4
6.3400000631809235
5
6.780000060796738
0
6.7800000

0
0
1
0
2
1.9199999570846558
3
3.0699999779462814
0
5.070000007748604
1
5.070000007748604
2
6.669999971985817
3
7.5899999886751175
0
7.5899999886751175
1
7.5899999886751175
2
7.5899999886751175
3
7.5899999886751175
0
7.5899999886751175
1
7.5899999886751175
2
7.5899999886751175
3
7.5899999886751175
0
0
1
0.23999999463558197
2
2.69999997317791
3
2.819999970495701
4
3.169999971985817
5
3.8299999684095383
0
5.529999986290932
1
5.529999986290932
2
7.579999968409538
3
7.659999966621399
4
7.939999967813492
5
8.48999996483326
0
8.48999996483326
1
8.48999996483326
2
8.48999996483326
3
8.48999996483326
4
8.48999996483326
5
8.48999996483326
0
8.48999996483326
1
8.48999996483326
2
8.48999996483326
3
8.48999996483326
4
8.48999996483326
5
8.48999996483326
0
0
1
0
2
1.3199999928474426
3
2.119999974966049
0
3.620000034570694
1
3.620000034570694
2
4.7200000286102295
3
5.360000014305115
0
5.360000014305115
1
5.360000014305115
2
5.360000014305115
3
5.360000014305115
0
5.360000014305115
1
5.36000001430511

0
0
1
0
2
2.8199999928474426
3
3.119999997317791
4
3.769999973475933
0
5.219999931752682
1
5.219999931752682
2
7.569999925792217
3
7.7699999287724495
4
8.289999909698963
0
8.289999909698963
1
8.289999909698963
2
8.289999909698963
3
8.289999909698963
4
8.289999909698963
0
8.289999909698963
1
8.289999909698963
2
8.289999909698963
3
8.289999909698963
4
8.289999909698963
0
0
1
0.03999999910593033
2
0.03999999910593033
3
1.2400000169873238
4
2.499999977648258
5
2.9499999955296516
6
3.009999994188547
0
4.959999922662973
1
4.959999922662973
2
4.959999922662973
3
5.959999937564135
4
6.799999911338091
5
7.159999925643206
6
7.2099999245256186
0
7.2099999245256186
1
7.2099999245256186
2
7.2099999245256186
3
7.2099999245256186
4
7.2099999245256186
5
7.2099999245256186
6
7.2099999245256186
0
7.2099999245256186
1
7.2099999245256186
2
7.2099999245256186
3
7.2099999245256186
4
7.2099999245256186
5
7.2099999245256186
6
7.2099999245256186
0
0
1
0
2
0.8400000035762787
3
1.1399999968707561
4
1.69999999925

0
0
1
0
2
1.9199999570846558
3
2.579999953508377
4
3.479999989271164
5
3.6599999852478504
0
5.31000005081296
1
5.31000005081296
2
6.910000015050173
3
7.350000012665987
4
8.070000041276217
5
8.220000037923455
0
8.220000037923455
1
8.220000037923455
2
8.220000037923455
3
8.220000037923455
4
8.220000037923455
5
8.220000037923455
0
8.220000037923455
1
8.220000037923455
2
8.220000037923455
3
8.220000037923455
4
8.220000037923455
5
8.220000037923455
0
0
1
2.940000057220459
2
3.290000058710575
3
3.5900000631809235
0
5.53999999165535
1
7.9900000393390656
2
8.270000040531158
3
8.520000044256449
0
8.520000044256449
1
8.520000044256449
2
8.520000044256449
3
8.520000044256449
0
8.520000044256449
1
8.520000044256449
2
8.520000044256449
3
8.520000044256449
0
0
1
0.23999999463558197
2
0.23999999463558197
3
2.4600000232458115
4
3.560000017285347
0
5.110000029206276
1
5.110000029206276
2
5.110000029206276
3
6.960000053048134
4
7.840000048279762
0
7.840000048279762
1
7.840000048279762
2
7.84000004827976

### Law

In [56]:
Law = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=6
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Law.append(weight)


0
0
1
0.6599999964237213
2
5.260000079870224
3
5.750000081956387
4
7.070000052452087
5
7.77000005543232
0
0
1
0.11999999731779099
2
5.6200001165270805
3
5.760000113397837
4
6.750000152736902
5
6.9100001491606236
0
0
1
0.1599999964237213
2
2.860000103712082
3
6.820000261068344
4
6.890000259503722
0
0
1
0.35999999195337296
2
0.3999999910593033
3
1.5999999642372131
4
3.909999892115593
5
4.069999888539314
0
0
1
0.11999999731779099
2
0.3200000002980232
3
1.7200000062584877
4
5.239999927580357
0
0
1
0.17999999597668648
2
0.29999999329447746
3
3.1000000052154064
4
4.419999975711107
5
4.559999972581863
0
0
1
0.20000000298023224
2
3.000000014901161
3
4.870000034570694
0
0
1
0.17999999597668648
2
0.3399999924004078
3
5.240000087767839
4
5.380000084638596
5
6.370000123977661
0
0
1
0.23999999463558197
2
2.5400000363588333
3
6.389999970793724
0
0
1
0.2800000011920929
2
5.1800000965595245
3
5.320000093430281
4
6.6400000639259815
0
0
1
0.5400000214576721
2
4.540000081062317
3
5.310000084340572
4
5.45

0
0
1
0.1599999964237213
2
5.459999710321426
3
5.739999704062939
4
6.949999697506428
0
0
1
0.6000000089406967
2
0.760000005364418
3
4.760000064969063
4
5.64000004529953
0
0
1
0.11999999731779099
2
0.19999999552965164
3
3.399999924004078
4
5.5999999567866325
0
0
1
0.47999998927116394
2
5.479999989271164
3
6.139999974519014
0
0
1
0.36000001430511475
2
3.7600000500679016
3
5.850000023841858
0
0
1
0.23999999463558197
2
0.35999999195337296
3
3.160000003874302
4
4.369999997317791
5
4.509999994188547
0
0
1
0.5400000214576721
2
0.6600000187754631
3
2.559999994933605
4
5.41999989002943
5
5.629999885335565
0
0
1
0.6000000089406967
2
5.200000092387199
3
5.620000082999468
4
6.940000053495169
5
7.640000056475401
0
0
1
0.1599999964237213
2
5.859999924898148
3
6.959999941289425
4
7.029999939724803
0
0
1
0.47999998927116394
2
0.5599999874830246
3
3.9600000232458115
4
4.100000020116568
5
5.9700000397861
0
0
1
0.4000000059604645
2
1.5
3
3.919999986886978
0
0
1
0.17999999597668648
2
0.29999999329447746
3

0
0
1
0.47999998927116394
2
0.5599999874830246
3
4.359999939799309
4
5.019999925047159
0
0
1
0.17999999597668648
2
0.3399999924004078
3
3.9400001354515553
4
4.010000133886933
5
5.110000150278211
0
0
1
0.11999999731779099
2
4.120000056922436
3
5.220000073313713
4
5.4000000692903996
0
0
1
0.07999999821186066
2
3.1800000220537186
3
3.460000015795231
4
6.210000015795231
0
0
1
0.1599999964237213
2
4.659999877214432
3
4.939999870955944
4
5.92999991029501
0
0
1
0.1599999964237213
2
6.659999758005142
3
6.869999753311276
4
7.30999974347651
0
0
1
0.20000000298023224
2
2.3999999910593033
3
4.0500000566244125
0
0
1
0.20000000298023224
2
4.299999967217445
3
4.719999957829714
4
6.259999964386225
5
6.329999962821603
0
0
1
0.3500000014901161
2
0.7700000032782555
3
1.0500000044703484
4
1.7500000074505806
5
2.240000009536743
6
3.0100000128149986
7
3.780000016093254
8
4.480000019073486
9
5.740000024437904
10
6.230000026524067
11
6.7200000286102295
12
7.280000030994415
13
7.700000032782555
14
8.2600000351

0
0
1
0.11999999731779099
2
0.23999999463558197
3
2.2400000244379044
4
5.0999999195337296
5
5.169999917969108
0
0
1
0.11999999731779099
2
3.0199999138712883
3
4.009999953210354
4
4.219999948516488
0
0
1
0.23999999463558197
2
4.939999982714653
3
5.079999979585409
4
6.070000018924475
0
0
1
0.3199999928474426
2
2.919999897480011
3
3.059999894350767
4
6.799999933689833
0
0
1
0.35999999195337296
2
0.5599999949336052
3
3.260000102221966
4
4.800000108778477
0
0
1
0.35999999195337296
2
0.5199999883770943
3
4.919999964535236
4
6.129999957978725
0
0
1
0.05999999865889549
2
0.17999999597668648
3
5.4799997098743916
4
5.619999706745148
5
6.9399996772408485
0
0
1
0.20000000298023224
2
2.7999999076128006
3
2.9399999044835567
4
5.689999904483557
5
5.759999902918935
0
0
1
0.20000000298023224
2
2.7999999076128006
3
2.9399999044835567
4
5.689999904483557
5
5.759999902918935
0
0
1
0.20000000298023224
2
4.899999991059303
3
5.24999999627471
4
6.679999943822622
0
0
1
0.05999999865889549
2
0.3399999998509884


### Health

In [57]:
Health = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=7
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Health.append(weight)


0
0
1
1.4299999922513962
2
6.950000092387199
3
6.950000092387199
4
7.430000081658363
5
7.710000082850456
0
9.41000010073185
1
10.400000095367432
2
13.160000145435333
3
13.160000145435333
4
13.640000134706497
5
13.92000013589859
0
0
1
0
2
6.6000001430511475
3
6.6000001430511475
4
6.960000157356262
5
7.200000151991844
0
10.100000068545341
1
10.100000068545341
2
13.400000140070915
3
13.400000140070915
4
13.76000015437603
5
13.900000151246786
0
0
1
0
2
3.2400001287460327
3
4.680000185966492
4
4.770000183954835
0
7.970000112429261
1
7.970000112429261
2
9.590000176802278
3
11.030000234022737
4
11.08000023290515
0
0
1
0.7799999825656414
2
0.7799999825656414
3
2.2199999503791332
4
3.0599999241530895
5
3.2999999187886715
0
9.099999751895666
1
9.639999739825726
2
9.639999739825726
3
10.359999723732471
4
11.199999697506428
5
11.339999694377184
0
0
1
0.25999999418854713
2
0.25999999418854713
3
1.9400000013411045
4
3.219999972730875
0
7.820000056177378
1
8.000000052154064
2
8.000000052154064
3
8.84

0
0
1
0
2
3.4799998998641968
3
4.279999911785126
0
8.679999887943268
1
8.679999887943268
2
10.419999837875366
3
11.219999849796295
0
0
1
0.12999999709427357
2
0.12999999709427357
3
2.1700000185519457
4
2.4500000197440386
0
9.449999900534749
1
9.539999898523092
2
9.539999898523092
3
10.559999909251928
4
10.839999910444021
0
0
1
0
2
4.800000071525574
3
4.800000071525574
4
5.400000095367432
5
5.490000093355775
0
8.790000224485993
1
8.790000224485993
2
11.19000026024878
3
11.19000026024878
4
11.790000284090638
5
11.840000282973051
0
0
1
0
2
3.8399999141693115
3
3.8399999141693115
4
5.039999961853027
0
8.139999985694885
1
8.139999985694885
2
10.059999942779541
3
10.059999942779541
4
11.259999990463257
0
0
1
0
2
2.6399999856948853
3
2.6399999856948853
4
3.439999997615814
5
3.709999991580844
6
3.949999986216426
0
8.149999855086207
1
8.149999855086207
2
9.46999984793365
3
9.46999984793365
4
10.269999859854579
5
10.419999856501818
6
10.559999853372574
0
0
1
2.1600000858306885
2
3.24000012874603

0
0
1
0
2
3.3600000143051147
3
3.3600000143051147
4
4.199999988079071
5
4.289999986067414
0
8.590000057592988
1
8.590000057592988
2
10.270000064745545
3
10.270000064745545
4
11.110000038519502
5
11.160000037401915
0
0
1
0
2
0.23999999463558197
3
0.8799999803304672
0
5.679999873042107
1
5.679999873042107
2
5.799999870359898
3
6.439999856054783
0
0
1
0
2
6.359999656677246
3
6.359999656677246
4
6.79999965429306
0
9.499999761581421
1
9.499999761581421
2
12.679999589920044
3
12.679999589920044
4
13.119999587535858
0
0
1
1.3000000193715096
2
1.3000000193715096
3
6.100000090897083
4
6.420000083744526
0
10.020000226795673
1
10.920000240206718
2
10.920000240206718
3
13.320000275969505
4
13.640000268816948
0
0
1
0.25999999418854713
2
0.25999999418854713
3
4.099999908357859
4
4.899999920278788
0
9.199999991804361
1
9.379999987781048
2
9.379999987781048
3
11.299999944865704
4
12.099999956786633
0
0
1
1.0399999767541885
2
7.0399999767541885
3
7.2799999713897705
0
10.779999911785126
1
11.49999989569

0
0
1
0.3899999912828207
2
0.3899999912828207
3
6.870000248774886
4
7.3100002463907
0
10.31000036559999
1
10.58000035956502
2
10.58000035956502
3
13.820000488311052
4
14.260000485926867
0
0
1
0
2
5.639999985694885
3
5.639999985694885
4
5.959999978542328
0
9.660000026226044
1
9.660000026226044
2
12.480000019073486
3
12.480000019073486
4
12.800000011920929
0
0
1
0
2
3.6000001430511475
3
4.680000185966492
0
8.680000245571136
1
8.680000245571136
2
10.48000031709671
3
11.560000360012054
0
0
1
0.3899999912828207
2
0.3899999912828207
3
5.550000077113509
4
5.990000074729323
0
9.889999931678176
1
10.159999925643206
2
10.159999925643206
3
12.73999996855855
4
13.179999966174364
0
0
1
0.3899999912828207
2
0.3899999912828207
3
3.2699999269098043
4
3.2699999269098043
5
4.549999898299575
0
8.349999850615859
1
8.619999844580889
2
8.619999844580889
3
10.05999981239438
4
10.05999981239438
5
11.339999783784151
0
0
1
1.4299999922513962
2
6.590000078082085
3
7.070000067353249
0
10.470000103116035
1
11.4600

0
0
1
0.3899999912828207
2
0.3899999912828207
3
2.069999998435378
4
3.1500000413507223
0
8.349999850615859
1
8.619999844580889
2
8.619999844580889
3
9.459999848157167
4
10.539999891072512
0
0
1
0
2
6.359999656677246
3
6.359999656677246
4
6.719999670982361
0
9.619999587535858
1
9.619999587535858
2
12.799999415874481
3
12.799999415874481
4
13.159999430179596
0
0
1
0.25999999418854713
2
0.25999999418854713
3
2.8999999798834324
4
3.6999999918043613
5
3.7899999897927046
0
8.789999989792705
1
8.969999985769391
2
8.969999985769391
3
10.289999978616834
4
11.089999990537763
5
11.139999989420176
0
0
1
0
2
4.919999957084656
3
4.919999957084656
4
5.719999969005585
5
5.899999964982271
0
8.399999964982271
1
8.399999964982271
2
10.859999943524599
3
10.859999943524599
4
11.659999955445528
5
11.759999953210354
0
0
1
2.2799999713897705
2
3.6799999475479126
0
8.280000030994415
1
9.4200000166893
2
10.819999992847443
0
0
1
1.0399999767541885
2
1.0399999767541885
3
5.5999999195337296
4
5.8399999141693115
0


0
0
1
0
2
3.8399999141693115
3
5.119999885559082
4
5.2999998815357685
0
8.300000000745058
1
8.300000000745058
2
10.219999957829714
3
11.499999929219484
4
11.59999992698431
0
0
1
0.3899999912828207
2
0.3899999912828207
3
4.589999919757247
4
5.629999881610274
0
8.529999798163772
1
8.799999792128801
2
8.799999792128801
3
10.899999756366014
4
11.939999718219042
0
0
1
0
2
4.440000057220459
3
4.440000057220459
4
4.880000054836273
0
8.880000114440918
1
8.880000114440918
2
11.100000143051147
3
11.100000143051147
4
11.540000140666962
0
0
1
0
2
4.800000071525574
3
4.800000071525574
4
5.440000057220459
0
8.740000188350677
1
8.740000188350677
2
11.140000224113464
3
11.140000224113464
4
11.78000020980835
0
0
1
0
2
3.6000001430511475
3
3.6000001430511475
4
4.800000190734863
0
8.00000011920929
1
8.00000011920929
2
9.800000190734863
3
9.800000190734863
4
11.000000238418579
0
0
1
0
2
3.960000157356262
3
3.960000157356262
4
4.280000150203705
0
9.380000054836273
1
9.380000054836273
2
11.360000133514404
3

3
11.530000109225512
4
11.780000112950802
0
0
1
0.12999999709427357
2
0.12999999709427357
3
4.090000154450536
4
4.090000154450536
5
4.650000156834722
6
4.830000152811408
0
8.73000000976026
1
8.820000007748604
2
8.820000007748604
3
10.800000086426735
4
10.800000086426735
5
11.36000008881092
6
11.460000086575747
0
0
1
3.6000001430511475
2
3.6000001430511475
3
4.040000140666962
4
4.490000147372484
0
9.290000040084124
1
11.090000111609697
2
11.090000111609697
3
11.530000109225512
4
11.780000112950802
0
0
1
0
0
0.7000000029802322
1
0.7000000029802322
0
0
1
3.4799998998641968
2
3.4799998998641968
3
3.9999998807907104
4
4.53999986872077
0
9.139999952167273
1
10.879999902099371
2
10.879999902099371
3
11.399999883025885
4
11.699999876320362
0
0
1
0.25999999418854713
2
0.25999999418854713
3
2.660000029951334
4
3.6999999918043613
5
3.7899999897927046
0
8.489999977871776
1
8.669999973848462
2
8.669999973848462
3
9.869999991729856
4
10.909999953582883
5
10.959999952465296
0
0
1
0
2
3.47999989986419

0
1
2.400000035762787
2
3.0000000596046448
0
9.499999821186066
1
10.699999839067459
2
11.299999862909317
0
0
1
0
2
4.800000071525574
3
4.800000071525574
4
5.64000004529953
0
8.64000016450882
1
8.64000016450882
2
11.040000200271606
3
11.040000200271606
4
11.880000174045563
0
0
1
0
2
1.4399999678134918
3
2.7199999392032623
0
8.019999653100967
1
8.019999653100967
2
8.739999637007713
3
10.019999608397484
0
0
1
0
2
2.1600000858306885
3
2.1600000858306885
4
3.080000102519989
0
8.279999911785126
1
8.279999911785126
2
9.35999995470047
3
9.35999995470047
4
10.27999997138977
0
0
1
0
2
4.800000071525574
3
5.880000114440918
0
8.280000060796738
1
8.280000060796738
2
10.680000096559525
3
11.760000139474869
0
0
1
0
2
5.639999985694885
3
6.119999974966049
4
6.209999972954392
0
9.30999999679625
1
9.30999999679625
2
12.129999989643693
3
12.609999978914857
4
12.65999997779727
0
0
1
0
2
1.559999942779541
3
2.319999933242798
0
7.319999933242798
1
7.319999933242798
2
8.099999904632568
3
8.859999895095825


In [58]:
Environment = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=8
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Environment.append(weight)


0
0
1
0.6599999964237213
2
0.6599999964237213
3
0.6599999964237213
4
0.6599999964237213
5
0.6599999964237213
0
0.6599999964237213
1
0.6599999964237213
2
0.6599999964237213
3
0.9399999976158142
4
0.9399999976158142
5
0.9399999976158142
0
0.9399999976158142
1
0.9399999976158142
2
0.9399999976158142
3
0.9399999976158142
4
0.9399999976158142
5
0.9399999976158142
0
0.9399999976158142
1
0.9399999976158142
2
0.9399999976158142
3
0.9399999976158142
4
0.9399999976158142
5
0.9399999976158142
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0.07999999821186066
4
0.07999999821186066
5
0.07999999821186066
0
0.07999999821186066
1
0.3499999921768904
2
0.3499999921768904
3
0.3499999921768904
4
0.3499999921768904
5
0.3499999921768904
0
0.3499999921768904
1
0.3499999921768904
2
0.3499999921768904
3
0.3499999921768904
4
0.3499999921768904
5
0.3499999921768904
0
0
1
0
2
0
3
0
4
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
0
0.059999

0
0
1
0
2
0
3
0
4
0
5
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.3799999915063381
4
0.3799999915063381
5
0.3799999915063381
0
0.3799999915063381
1
0.5599999874830246
2
0.5599999874830246
3
0.5599999874830246
4
0.5599999874830246
5
0.5599999874830246
0
0.5599999874830246
1
0.5599999874830246
2
0.5599999874830246
3
0.5599999874830246
4
0.5599999874830246
5
0.5599999874830246
0
0
1
0.11999999731779099
2
0.11999999731779099
3
0.11999999731779099
4
0.11999999731779099
5
0.11999999731779099
0
0.11999999731779099
1
0.11999999731779099
2
0.11999999731779099
3
0.11999999731779099
4
0.1599999964237213
5
0.1599999964237213
0
0.1599999964237213
1
0.1599999964237213
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
5
0.6999999843537807
0
0.6999999843537807
1
0.6999999843537807
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
5
0.6999999843537807
0
0
1
0
2
0
3
0
4
0
5
0.11999999731779099
0
0.11999999731779099
1
0.1199999973177

0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0.07999999821186066
4
0.07999999821186066
0
0.07999999821186066
1
0.6199999861419201
2
0.6199999861419201
3
0.6199999861419201
4
0.6199999861419201
0
0.6199999861419201
1
0.6199999861419201
2
0.6199999861419201
3
0.6199999861419201
4
0.6199999861419201
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0.07999999821186066
4
0.07999999821186066
0
0.07999999821186066
1
0.7999999821186066
2
0.7999999821186066
3
0.7999999821186066
4
0.7999999821186066
0
0.7999999821186066
1
0.7999999821186066
2
0.7999999821186066
3
0.7999999821186066
4
0.7999999821186066
0
0
1
0.23999999463558197
2
0.23999999463558197
3
0.23999999463558197
4
0.23999999463558197
0
0.23999999463558197
1
0.23999999463558197
2
0.23999999463558197
3
0.23999999463558197
4
0.23999999463558197
0
0.23999999463558197
1
0.23999999463558197
2
0.5999999865889549
3
0.5999999865889549
4
0.5999999865889549
0
0.5999999865889549
1
0.5999999865889549
2
0.5999999865889549
3
0.5999999865889549
4
0.5999999865889549
0
0
1
0
2
0


0
0
1
0.11999999731779099
2
0.11999999731779099
3
0.11999999731779099
4
0.11999999731779099
5
0.11999999731779099
0
0.11999999731779099
1
0.11999999731779099
2
0.11999999731779099
3
0.11999999731779099
4
0.11999999731779099
5
0.11999999731779099
0
0.11999999731779099
1
0.11999999731779099
2
0.29999999329447746
3
0.29999999329447746
4
0.29999999329447746
5
0.29999999329447746
0
0.29999999329447746
1
0.29999999329447746
2
0.29999999329447746
3
0.29999999329447746
4
0.29999999329447746
5
0.29999999329447746
0
0
1
0
2
0
3
0
4
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
0
0.05999999865889549
1
0.23999999463558197
2
0.23999999463558197
3
0.23999999463558197
4
0.23999999463558197
0
0.23999999463558197
1
0.23999999463558197
2
0.23999999463558197
3
0.23999999463558197
4
0.23999999463558197
0
0
1
0.23999999463558197
2
0.23999999463558197
3
0.23999999463558197
4
0.23999999463558197
0
0.23999999463558197
1
0.2399

0
0
1
0
2
0
3
0
4
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
0
0.05999999865889549
1
0.41999999061226845
2
0.41999999061226845
3
0.41999999061226845
4
0.41999999061226845
0
0.41999999061226845
1
0.41999999061226845
2
0.41999999061226845
3
0.41999999061226845
4
0.41999999061226845
0
0
1
0.47999998927116394
2
0.47999998927116394
3
0.47999998927116394
4
0.47999998927116394
5
0.47999998927116394
0
0.47999998927116394
1
0.47999998927116394
2
0.47999998927116394
3
0.47999998927116394
4
0.5599999874830246
5
0.5599999874830246
0
0.5599999874830246
1
0.5599999874830246
2
0.7399999834597111
3
0.7399999834597111
4
0.7399999834597111
5
0.7399999834597111
0
0.7399999834597111
1
0.7399999834597111
2
0.7399999834597111
3
0.7399999834597111
4
0.7399999834597111
5
0.7399999834597111
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.9000000134110451
2
0.9000000134110451
3
0.9000000134110451
0
0.9000000134110451
1
0.900000013411

...

0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.35999999195337296
2
0.35999999195337296
3
0.35999999195337296
0
0.35999999195337296
1
0.35999999195337296
2
0.35999999195337296
3
0.35999999195337296
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0.1599999964237213
4
0.1599999964237213
0
0.1599999964237213
1
0.6999999843537807
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
0
0.6999999843537807
1
0.6999999843537807
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.17999999597668648
2
0.17999999597668648
3
0.17999999597668648
4
0.17999999597668648
0
0.17999999597668648
1
0.17999999597668648
2
0.17999999597668648
3
0.17999999597668648
4
0.17999999597668648
0
0
1
0
2
0
3
0
4
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
0
0.05999999865889549
1
0.5999999865889549
2
0.5999999865889549
3
0.5999999865889549
4
0.5999999865889549
0
0.5999999865889549
1
0.59999

0
0
1
0
2
0
3
0
4
0.05999999865889549
5
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
5
0.05999999865889549
0
0.05999999865889549
1
0.3299999926239252
2
0.3299999926239252
3
0.3299999926239252
4
0.3299999926239252
5
0.3299999926239252
0
0.3299999926239252
1
0.3299999926239252
2
0.3299999926239252
3
0.3299999926239252
4
0.3299999926239252
5
0.3299999926239252
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.630000002682209
2
0.630000002682209
3
0.630000002682209
0
0.630000002682209
1
0.630000002682209
2
0.630000002682209
3
0.630000002682209
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
7.740000128746033
3
7.740000128746033
4
7.740000128746033
0
7.740000128746033
1
7.740000128746033
2
7.740000128746033
3
7.740000128746033
4
7.740000128746033
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0.07999999821186066
4
0.07999999821186066
5
0.07999999821186066
0
0.07999999821186066
1
0.3499999921768904
2
0.3499999921768904
3
0.3

1
0
2
0
3
0
4
0.30000000447034836
0
0.30000000447034836
1
0.30000000447034836
2
0.5399999991059303
3
0.5399999991059303
4
0.5399999991059303
0
0.5399999991059303
1
0.5399999991059303
2
0.5399999991059303
3
0.5399999991059303
4
0.5399999991059303
0
0.5399999991059303
1
0.5399999991059303
2
0.5399999991059303
3
0.5399999991059303
4
0.5399999991059303
0
0
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
5
0.05999999865889549
6
0.17999999597668648
0
0.17999999597668648
1
0.17999999597668648
2
0.17999999597668648
3
0.17999999597668648
4
0.29999999329447746
5
0.29999999329447746
6
0.29999999329447746
0
0.29999999329447746
1
0.29999999329447746
2
1.0199999772012234
3
1.0199999772012234
4
1.0199999772012234
5
1.0199999772012234
6
1.0199999772012234
0
1.0199999772012234
1
1.0199999772012234
2
1.0199999772012234
3
1.0199999772012234
4
1.0199999772012234
5
1.0199999772012234
6
1.0199999772012234
0
0
1
0
2
0
3
0
4
0.30000000447034836
0
0.30000000447034836
1
0

0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.08999999798834324
2
0.08999999798834324
3
0.08999999798834324
0
0.08999999798834324
1
0.08999999798834324
2
0.08999999798834324
3
0.08999999798834324
0
0
1
0.17999999597668648
2
0.17999999597668648
3
0.17999999597668648
4
0.17999999597668648
0
0.17999999597668648
1
0.17999999597668648
2
0.17999999597668648
3
0.17999999597668648
4
0.17999999597668648
0
0.17999999597668648
1
0.17999999597668648
2
0.4499999899417162
3
0.4499999899417162
4
0.4499999899417162
0
0.4499999899417162
1
0.4499999899417162
2
0.4499999899417162
3
0.4499999899417162
4
0.4499999899417162
0
0
1
0
2
0
3
0
4
0.05999999865889549
0
0.05999999865889549
1
0.05999999865889549
2
0.05999999865889549
3
0.05999999865889549
4
0.05999999865889549
0
0.05999999865889549
1
0.6900000013411045
2
0.6900000013411045
3
0.6900000013411045
4
0.6900000013411045
0
0.6900000013411045
1
0.6900000013411045
2
0.6900000013411045
3
0.6900000013411045
4
0.6900000013411045
0
0
1
0.42000000178813934
2
0.4200000

0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0.1599999964237213
4
0.1599999964237213
0
0.1599999964237213
1
0.6999999843537807
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
0
0.6999999843537807
1
0.6999999843537807
2
0.6999999843537807
3
0.6999999843537807
4
0.6999999843537807
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.2699999939650297
2
0.2699999939650297
3
0.2699999939650297
0
0.2699999939650297
1
0.2699999939650297
2
0.2699999939650297
3
0.2699999939650297
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0.07999999821186066
4
0.07999999821186066
0
0.07999999821186066
1
0.5300000049173832
2
0.5300000049173832
3
0.5300000049173832
4
0.5300000049173832
0
0.5300000049173832
1
0.5300000049173832
2
0.5300000049173832
3
0.5300000049173832
4
0.5300000049173832
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.630000002682209
2
0.630000002682209
3
0.630000002682209
0
0.630000002682209
1
0.630000002682209
2
0.630000002682209
3
0.630000002682209
0
0
1
0
2
0
3
0
4
0.05

### Consitution

In [59]:
Constitution = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=9
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Constitution.append(weight)

0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0


3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0


0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0


### Resources

In [62]:
resource = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=10
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    resource.append(weight)

0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.4399999976158142
2
2.2800000309944153
3
2.770000033080578
4
3.250000022351742
5
3.600000023841858
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
2.200000047683716
3
2.340000044554472
4
2.7000000588595867
5
2.7800000570714474
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.0800000429153442
3
2.5200001001358032
4
2.620000097900629
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.23999999463558197
2
0.23999999463558197
3
0.7199999839067459
4
1.5599999576807022
5
1.6399999558925629
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.07999999821186066
2
0.07999999821186066
3
0.6400000005960464
4
1.919999971985817
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.2400000020861626
4
1.7199999913573265
5
1.9199999868869781
0
0
1
0
2
0
3
0

0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.11999999731779099
2
0.11999999731779099
3
0.7599999830126762
4
1.9999999925494194
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
2.119999885559082
3
2.18999988399446
4
2.7099998649209738
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.47999998927116394
2
0.47999998927116394
3
0.9199999868869781
4
2.679999977350235
5
2.8799999728798866
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.20000000298023224
2
0.20000000298023224
3
1.9200000315904617
4
2.060000028461218
5
2.7800000570714474
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0.07999999821186066
2
0.07999999821186066
3
1.2399999648332596
4
1.3099999632686377
5
2.029999991878867
6
2.129999989643693
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.9199999570846558
3
2.059999953955412
4
2.65999997779727
0
0
1
0
2
0
3
0


0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
1.0800000429153442
3
1.7200000286102295
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0.7200000286102295
3
1.9200000762939453
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0.7599999904632568
3
2.1200000047683716
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0.9599999785423279
3
2.35999995470047
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.9200000166893005
2
1.9599999785423279
3
2.2599999718368053
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
1.0399999618530273
3
1.5999999642372131
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0.47999998927116394
3
2.039999932050705
4
2.1399999298155308
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.2400000020861626
4
1.5200000032782555
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.11999999731779099
2
1.6000000163912773
3
2.6000000163912773


0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.07999999821186066
2
0.07999999821186066
3
0.919999971985817
4
2.000000014901161
5
2.200000010430813
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0.800000011920929
2
1.7200000286102295
3
2.22000003606081
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.7999999523162842
3
2.1999999582767487
4
2.239999957382679
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0.9200000166893005
3
0.9900000151246786
4
2.4300000723451376
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.3199999928474426
2
0.3199999928474426
3
1.5600000023841858
4
2.0799999833106995
5
2.11999998241663
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0.8399999737739563
2
2.8799999356269836
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.720000021159649
4
2.2100000232458115
5
2.450000017

0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.3200000524520874
3
1.5300000477582216
4
1.8500000406056643
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.07999999821186066
2
0.07999999821186066
3
0.919999971985817
4
1.9599999338388443
5
2.159999929368496
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.03999999910593033
2
0.7999999895691872
3
1.6399999633431435
4
2.2399999499320984
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
1.840000033378601
3
2.200000047683716
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0.8799999952316284
3
2.240000009536743
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0.800000011920929
2
2.800000011920929
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.23999999463558197
2
0.23999999463558197
3
1.799999937415123
4
1.869999935850501
5
2.3899999167770147
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.07999

0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
1.600000023841858
3
1.8100000191479921
4
2.41000004298985
5
2.510000040754676
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.2799999713897705
3
2.559999942779541
4
2.7599999383091927
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.1599999964237213
2
0.1599999964237213
3
1.280000001192093
4
1.9199999868869781
5
2.11999998241663
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0.5600000023841858
2
0.5600000023841858
3
1.9599999785423279
4
2.169999973848462
5
2.5699999798089266
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0.9200000166893005
3
2.64000004529953
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0.11999999731779099
2
0.11999999731779099
3
1.3200000450015068
4
1.760000042617321
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
1.7200000286102295
3
2.48000001907348

In [63]:
Rural = []
for k in range(len(d)):
    document = d['Manifesto'][k]
    tokens = word_tokenize(document)
    topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=35)

    x = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])
    
    weight = 0
    p=11
    for u in range(len(topa[p])):
        name = topa[p][u]
            
        for i in range(len(x)):
            print(i)
            print(weight)
            topics_word = x['words in topic'][i].split('+')
            for j in range(len(topics_word)):
                index = topics_word[j].find(name)
                if index!=-1:
                    weight+= float(topics_word[j].split('*')[0].split('.')[1])*float(x['weight'][i])
    Rural.append(weight)

0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0


0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
0
0
1
0
2
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0
0
0
1
0
2
0
3
0
4
0
5
0


In [65]:
df['Education'] = Education
df['Business'] = Business
df['Jobs & Economy'] = Economy
df['Labour relations'] = Labour
df['Tax'] = Tax
df['Security'] = Security
df['Judiciary Law'] = Law
df['Constitutional rights'] = Constitution
df['Rural life'] = Rural
df['Health & family'] = Health
df['Environment'] = Environment
df['Resources'] = resource

In [150]:
df.to_csv('USA_Topic_Modelling.csv',index=None)

# Final data